In [ ]:
]activate .

In [ ]:
]instantiate

In [ ]:
using SymPy
using Plots
using Interact
theme(:dark)
using Logging; Logging.disable_logging(Logging.Info);

In [ ]:
# Define the parameters with initial values

# Parameters
x₀ = 0             # Left limit (m)
L = 1              # Right limit (m)
step = 0.01        # Step
range = x₀:step:L  # Range
N_max = 15         # Maximum number of iterations
iter_fps = 3       # Frames per second for iterations
wave_fps = 30      # Frames per second for the wave
Δt = 6             # seconds

# Fourier coefficient and function
@syms n::(integer,positive), x::(real,positive), N::(integer,positive)
b = 8 / sympy.pi^2 * (-1)^n / (2n + 1)^2
g = b * sympy.sin((2n + 1) * sympy.pi * x / L)

# y component
@syms t::(real,positive), d(), k::(integer,positive), v::(integer,positive)
y = g * d(t)

# Wave function
wave_function = Eq(diff(y, (t, 2)) - v^2 * diff(y, (x, 2)) + k * diff(y, t), 0)

# Differential solve
N_wave = 1
solution = dsolve(wave_function, d(t), ics = Dict(d(0) => 1, diff(d(t), t)(0) => 1))

y = sympy.re(subs(y, d(t), solution.rhs))
# Extract the real part of the wave function
y_function = lambdify(y, [x, t, k, v, n])

# Wave animation function
@manipulate for N_in in dropdown(0:15, value=1, label="N"), k_in in dropdown(0:5, value=2, label="k (N/m)"), v_in in dropdown(1:5, value=2, label="v (m/s)")
    @time wave = @animate for current_t = 0:1/wave_fps:Δt
        print("\r")
        print("Completed: $(Int(round(current_t*100/Δt)))%")
        cord = zeros(length(range))
        for i = 0:N_in
            cord += y_function.(range, current_t, k_in, v_in, i)
        end
        plot(
            range,
            cord,
            grid = false,
            framestyle = :semi,
            linewidth = 2,
            foreground_color_legend = nothing,
            label = "N = $N_in; v = $v_in m/s; k = $k_in N/m; t = $(round(current_t; digits=1)) s",
        )
        xaxis!((x₀, L), 0:1:2*L)
        xlabel!("Length (m)")
        yaxis!((-1.5, 1.5), -2*L:1:2*L)
        ylabel!("Amplitude (m)")
    end
    gif(wave, "gifs/string_animation.gif", fps = wave_fps)
end
